# Simple Wing with Steady Aerodynamics
This is a short demonstration on how to use the Vortex Lattice Method (VLM) at the example of a simple wing.

It is also assumed that you sucessfully installed the software as a Python package as described in the [README](https://github.com/DLR-AE/PanelAero?tab=readme-ov-file#installation--use).

Let's see if we can import the software, which is a good indictor for a sucessful installation. In addition to the VLM inself, this tutorial also uses numpy and some helper functions provided along with the tutorials.

In [ ]:
import numpy as np

from panelaero import VLM

from tutorials.helper_functions import build_aeromodel, plotting

## Set-up of Geometry
The geometry of the lifting surfaces is typically described using CAERO1 cards in Nastran eight-character-notation. Each field has exactly 8 characters and the format is defined in the following way:
```
$------><------><------><------><------><------><------><------><------>
CAERO1  EID             CP      NSPAN   NCHORD                          +  
+       X1      Y1      Z1      X12     X4      Y4      Z4      X34
```
With:

EID = Element identification number  
CP  = Coordinate system  
NSPAN = Number of spanwise panels  
NCHORD = Number of chordwise panels  
X1, Y1, Z1 = Location of wing root leading edge  
X4, Y4, Z4 = Location of wing tip leading edge  
X12, X34 = Chord length at wing root and tip  

Note that Nastran allows more fields / options on this card (see Nastran Quick Reference Guide), but not all of them are needed for this example or are not supported and skipped by the reader.


This is the CAERO1 input card for a simple, rectangular wing with a wing span of 1.104 m, measured from tip to tip, and a chord length of 0.1 m. The wing is discretized with 40x10 panels.
```
$------><------><------><------><------><------><------><------><------>
CAERO1   6401001    1001       0      40      10                       1+       
+            0.0  -0.552     0.0     0.1     0.0  +0.552     0.0     0.1
```

A simple reader (build_aeromodel.py) is provied along with the examples, which is now used to parse the CAERO1 card.

In [ ]:
model = build_aeromodel.AeroModel('./simplewing.CAERO1')
model.build_aerogrid()

The result is a Python dictionary, which contains the geometrical information of all 400 aerodynamic panels. If you have Mayavi installed, the aerogrid can be visualized using the plotting routines (plotting.py) provided along with the examples.

In [ ]:
plots = plotting.DetailedPlots(model)
plots.plot_aerogrid(embed_in_notebook=True)

## Calculate the AIC Matrix
Based on a given aerodynamic grid, the Vortex Lattice Method (VLM) provided in this software calculates a matrix of so-called aerodynamic influence coefficients (AIC), which depends only on the geometry and the Mach number. The $\mathbf{AIC}$ matrix then relates an induced downwash $\mathbf{w_j}$ on each aerodynamic panel to a circulation strength $\Gamma_j$, which is translated to a pressure coefficient $c_p$.
$$\mathbf{\Delta c_p} = \mathbf{AIC}(Ma) \cdot \mathbf{w_j}$$
The $\mathbf{AIC}$ matrix (called Qjj in the following) is obtained in the following way:

In [ ]:
Qjj, _ = VLM.calc_Qjj(model.aerogrid, Ma=0.0)

Next, let's assume an onflow condition with Vtas = 25.0 m/s and an induced downwash of wj = 2.18 m/s on every panels, which corresponds to an angle of attack = 5.0 deg. Note that the downwash is scaled with Vtas.

In [ ]:
Vtas = 25.0
wj = np.ones(model.aerogrid['n']) * 2.18 / Vtas

Following the formula given above, multiplication of the downwash vector $\mathbf{w_j}$ with the $\mathbf{AIC}$ matrix yields the pressure coefficient distribution $\mathbf{\Delta c_p}$

In [ ]:
cp = Qjj.dot(wj)

## Vizualisation of Results
The pressure coefficient distribution may be visualized on the geometry using the helper function from above.

Advanced option: To get an interactive 3D plot, use 'embed_in_notebook=False', which renders the plot in a new window.

In [ ]:
plots.plot_aerogrid(cp, embed_in_notebook=True)

For many applications, including aeroelasticity, not nly the pressure distribution but the aerodynamic force vector is needed. From the pressure coefficient distribution cp, the aerodynamic forces Fxyz can be calculated by multiplication with area A and the normal vector of every panels as well as the dynamic pressure. 

In [ ]:
q_dyn = 1.225 / 2.0 * Vtas ** 2
Fxyz = q_dyn * model.aerogrid['N'].T * model.aerogrid['A'] * cp

The force vector Fxyz has the shape 3 x 400, reflecting the aerodynamic forces in x-, y- and z-direction per panel. Calculating the sums from all panels,  the aerodynamic lift created by the wing should be Fz = 18.47 N. The componetns Fx and Fy should be zero in this case, because the wing has no dihedral and the VLM calculates no drag.

In [ ]:
Fxyz.sum(axis=1)

## Summary
This tutorial demonstrated how to discretize a simple wing with aerodynamic panels and how to calculate an $\mathbf{AIC}$ matrix. After that, we vizualized the resulting pressure distribution and explored how to calculate a force vector from the pressures. 

For more information on the theoretical background of the VLM, please consult the Technical Report DLR-IB-AE-GO-2020-137.